<a href="https://colab.research.google.com/github/Tripthi0203/IIITH-NLP-Final-Project/blob/main/Spoken_Digit_Recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!wget https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/heads/master.zip
! unzip master.zip 


--2022-04-10 14:39:29--  https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/heads/master.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Jakobovski/free-spoken-digit-dataset/zip/refs/heads/master [following]
--2022-04-10 14:39:29--  https://codeload.github.com/Jakobovski/free-spoken-digit-dataset/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 52.193.111.178
Connecting to codeload.github.com (codeload.github.com)|52.193.111.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [        <=>         ]  15.66M  8.57MB/s    in 1.8s    

2022-04-10 14:39:31 (8.57 MB/s) - ‘master.zip’ saved [16421327]

Archive:  master.zip
e10dc3e12d6048514d72e3614a2a013ff2bc4dc8
   creating: free-sp

In [ ]:

import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
import numpy as np
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical


In [ ]:
def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
  sample_rate, samples = wav.read(audio_path)
  fig = plt.figure()
  fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
  ax = plt.Axes(fig, [0., 0., 1., 1.])
  ax.set_axis_off()
  fig.add_axes(ax)
  ax.specgram(samples, cmap=cmap, Fs=2, noverlap=noverlap)
  ax.xaxis.set_major_locator(plt.NullLocator())
  ax.yaxis.set_major_locator(plt.NullLocator())
  fig.savefig(save_path, bbox_inches="tight", pad_inches=0)
  

In [ ]:
def dir_to_spectrogram(audio_dir, spectrogram_dir, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
  file_names = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f)) and '.wav' in f]
  for file_name in file_names:
    print(file_name)
    audio_path = audio_dir + file_name
    spectogram_path = spectrogram_dir + file_name.replace('.wav', '.png')
    wav_to_spectrogram(audio_path, spectogram_path, spectrogram_dimensions=spectrogram_dimensions, noverlap=noverlap, cmap=cmap)
    

In [ ]:
audio_folder = "recordings/" # move the recordings folder from free-spoken-digit-dataset to outside
spectrogram_folder = "spectrograms/"  # create a new folder called "spectrograms"
dir_to_spectrogram(audio_folder, spectrogram_folder)


In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical


In [ ]:
imagesDir = "spectrograms/"
trainset = []
testset = []
for file in os.listdir(imagesDir):
  label = file.split('_')[0]
  sample_number = file.split('_')[2]
  img = image.load_img(imagesDir+file)
  if sample_number in ['0.png','1.png','2.png','3.png','4.png']:
    testset.append([image.img_to_array(img), label])
  else:
    trainset.append([image.img_to_array(img), label])


In [ ]:

X_train = [item[0] for item in trainset]
y_train = [item[1] for item in trainset]
X_test = [item[0] for item in testset]
y_test = [item[1] for item in testset]


In [ ]:
X_train = np.asanyarray(X_train)
y_train = np.asanyarray(y_train)
X_test = np.asanyarray(X_test)
y_test = np.asanyarray(y_test)


In [ ]:

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train /= 255
X_test /= 255








In [ ]:
#CNN MODEL

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras import models


In [ ]:
data_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
def basic_cnn():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=data_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy'])
  return model
  

In [ ]:
model0 = basic_cnn()
model0.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 63, 63, 32)        416       
                                                                 
 batch_normalization (BatchN  (None, 63, 63, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 48)        6192      
                                                                 
 batch_normalization_1 (Batc  (None, 62, 62, 48)       192       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 61, 61, 120)       23160     
                                                                 
 batch_normalization_2 (Batc  (None, 61, 61, 120)      4

In [ ]:
model0.fit(X_train, y_train, batch_size = 50, validation_split=0.2, epochs = 100, verbose = 1)


Epoch 1/100
44/44 [==============================] - 85s 2s/step - loss: 3.2338 - accuracy: 0.1306 - val_loss: 2.3320 - val_accuracy: 0.0963
Epoch 2/100
44/44 [==============================] - 82s 2s/step - loss: 2.8390 - accuracy: 0.1824 - val_loss: 2.3801 - val_accuracy: 0.0870
Epoch 3/100
44/44 [==============================] - 82s 2s/step - loss: 2.5084 - accuracy: 0.2403 - val_loss: 2.4321 - val_accuracy: 0.0833
Epoch 4/100
44/44 [==============================] - 82s 2s/step - loss: 2.2794 - accuracy: 0.2926 - val_loss: 2.4874 - val_accuracy: 0.0833
Epoch 5/100
44/44 [==============================] - 82s 2s/step - loss: 2.0715 - accuracy: 0.3292 - val_loss: 2.5497 - val_accuracy: 0.0944
Epoch 6/100
44/44 [==============================] - 82s 2s/step - loss: 1.9476 - accuracy: 0.3644 - val_loss: 2.5761 - val_accuracy: 0.1093
Epoch 7/100
44/44 [==============================] - 82s 2s/step - loss: 1.8414 - accuracy: 0.3991 - val_loss: 2.5068 - val_accuracy: 0.1315
Epoch 8/100
4

In [ ]:
model0.evaluate(X_test, y_test)

10/10 [==============================] - 3s 251ms/step - loss: 0.2567 - accuracy: 0.9300


[0.2567388117313385, 0.9300000071525574]

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format
model0.save("spoken_digit_recognition_.h5")

In [ ]:
index = 5 # we can specify any index that is in test data.the model predicts the crt o/p.
print('Original value' ,np.argmax(y_test[index]))
print('Prediction' ,np.argmax(model0.predict(X_test[index].reshape(1,64,64,3))))

Original value 4
Prediction 4
